In [1]:
!pip -q install -U keras-tuner gputil

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [15]:
import os, json, time, platform
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt
import GPUtil
from google.colab import drive
from keras import ops as K

In [2]:
# Colab: mount Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
# Paths
DATA_DIR = "/content/drive/MyDrive/EdgeMeter_AIv2/data"
WIN_KEY  = "48to12"
WIN_TAG  = "48_12"

HP_PATH  = os.path.join(DATA_DIR, f"best_liteformer_hp_{WIN_KEY}.json")
LOG_PATH = os.path.join(DATA_DIR, f"liteformer_tuning_log_{WIN_KEY}.json")

# Reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [4]:
# GPU info
gpus = tf.config.list_physical_devices('GPU')
print("GPUs visible to TF:", gpus)
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except Exception:
        pass
gpu_name = GPUtil.getGPUs()[0].name if GPUtil.getGPUs() else "None"
print("GPU in use:", gpu_name)

GPUs visible to TF: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU in use: NVIDIA A100-SXM4-40GB


In [5]:
# Loading Data (48-12)
def _path(d, base, tag):
    p1 = os.path.join(d, f"{base}_{tag}.npy")
    p0 = os.path.join(d, f"{base}.npy")
    if os.path.exists(p1): return p1
    if os.path.exists(p0): return p0
    raise FileNotFoundError(f"Missing {base}_{tag}.npy (or {base}.npy) in {d}")

X_train = np.load(_path(DATA_DIR, "X_train", WIN_TAG))
y_train = np.load(_path(DATA_DIR, "y_train", WIN_TAG))
X_val   = np.load(_path(DATA_DIR, "X_val",   WIN_TAG))
y_val   = np.load(_path(DATA_DIR, "y_val",   WIN_TAG))
X_test  = np.load(_path(DATA_DIR, "X_test",  WIN_TAG))
y_test  = np.load(_path(DATA_DIR, "y_test",  WIN_TAG))

In [6]:
# Shape + NaN checks
assert X_train.ndim==3 and X_val.ndim==3 and X_test.ndim==3, "X must be (N,T,F)"
assert y_train.ndim==2 and y_val.ndim==2 and y_test.ndim==2, "y must be (N,12)"
assert y_train.shape[1]==12 and y_val.shape[1]==12 and y_test.shape[1]==12, "y must have 12 steps"
assert X_train.shape[1]==48, f"Expected T=48; got {X_train.shape[1]}"
for name, arr in [("X_train",X_train),("X_val",X_val),("X_test",X_test),
                  ("y_train",y_train),("y_val",y_val),("y_test",y_test)]:
    if np.isnan(arr).any():
        raise ValueError(f"NaNs detected in {name}")

timesteps  = X_train.shape[1]  # 48
n_features = X_train.shape[2]
out_steps  = y_train.shape[1]  # 12

print(f"Train X: {X_train.shape} | y: {y_train.shape}")
print(f"Val   X: {X_val.shape}   | y: {y_val.shape}")
print(f"Test  X: {X_test.shape}  | y: {y_test.shape}")

Train X: (10719826, 48, 11) | y: (10719826, 12)
Val   X: (3072784, 48, 11)   | y: (3072784, 12)
Test  X: (1536392, 48, 11)  | y: (1536392, 12)


In [7]:
# 5% subset for fast tuning
subset_frac   = 0.05
n_train_small = max(1, int(subset_frac * X_train.shape[0]))
n_val_small   = max(1, int(subset_frac * X_val.shape[0]))

X_train_small = X_train[:n_train_small]
y_train_small = y_train[:n_train_small]
X_val_small   = X_val[:n_val_small]
y_val_small   = y_val[:n_val_small]

print(f"Subset shapes → X_train_small: {X_train_small.shape} | X_val_small: {X_val_small.shape}")


Subset shapes → X_train_small: (535991, 48, 11) | X_val_small: (153639, 48, 11)


In [16]:
# LiteFormer
def slice_patches(x, patch_size):
    # (B,T,F) → (B,P,patch_size*F), P = floor(T/patch_size)
    T = tf.shape(x)[1]
    F = tf.shape(x)[2]
    t_trim = (T // patch_size) * patch_size
    x = x[:, :t_trim, :]
    P = t_trim // patch_size
    return tf.reshape(x, [tf.shape(x)[0], P, patch_size * F])

def build_liteformer_v1(hp):
    """
    EXACT v1 LiteFormer block (do not change):
      - slice patches
      - Dense(hidden_dim)
      - + learnable positional embedding
      - Dense(hidden_dim, gelu) + Dropout
      - Linear attention via QKV (softmax(QK^T/sqrt(d))V) + Dropout
      - Residual + LayerNorm
      - FFN: Dense(2*hidden_dim, gelu) -> Dense(hidden_dim) + LayerNorm
      - GAP -> Dense(12)
    """
    patch_size    = hp.Choice('patch_size', [4, 6, 8, 12])
    hidden_dim    = hp.Int('hidden_dim', 32, 128, step=16)
    dropout       = hp.Float('dropout', 0.0, 0.3, step=0.05)
    learning_rate = hp.Float('learning_rate', 1e-4, 5e-3, sampling='log')

    inputs = keras.Input(shape=(timesteps, n_features))
    x = layers.Lambda(lambda t: slice_patches(t, patch_size))(inputs)   # (B,P,patch*F)

    # Project to hidden_dim
    x = layers.Dense(hidden_dim)(x)

    # Positional embedding (learnable)
    num_patches = timesteps // patch_size
    pos_embed = layers.Embedding(input_dim=num_patches, output_dim=hidden_dim)
    positions = tf.range(start=0, limit=num_patches, delta=1)
    pos_encoding = pos_embed(positions)         # (P, hidden_dim)
    x = x + pos_encoding                        # broadcast over batch

    # Pre-attn transform
    x = layers.Dense(hidden_dim, activation='gelu')(x)
    x = layers.Dropout(dropout)(x)

    # Linear attention (QKV)
    q = layers.Dense(hidden_dim)(x)
    k = layers.Dense(hidden_dim)(x)
    v = layers.Dense(hidden_dim)(x)
    scale = K.sqrt(K.cast(hidden_dim, "float32"))

    attn_scores  = K.matmul(q, K.swapaxes(k, -1, -2)) / scale
    attn_weights = K.softmax(attn_scores, axis=-1)
    attn_output  = K.matmul(attn_weights, v)

    # Residual + Norm
    x = layers.Add()([x, attn_output])
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # FFN + Norm
    x = layers.Dense(hidden_dim * 2, activation='gelu')(x)
    x = layers.Dense(hidden_dim)(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)

    # Head
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(out_steps)(x)

    model = keras.Model(inputs, outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse',
        metrics=['mae']
    )
    return model

In [17]:
# Tuner (BayesianOptimization)
tuner = kt.BayesianOptimization(
    build_liteformer_v1,
    objective="val_loss",
    max_trials=20,
    directory=DATA_DIR,
    project_name=f"liteformer_retune_{WIN_KEY}"
)

early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=4, restore_best_weights=True
)

device = "/GPU:0" if tf.config.list_physical_devices("GPU") else "/CPU:0"
print("Using device:", device)

start_time = time.time()
with tf.device(device):
    tuner.search(
        X_train_small, y_train_small,
        validation_data=(X_val_small, y_val_small),
        epochs=50,
        batch_size=512,
        callbacks=[early_stop],
        verbose=1
    )
end_time = time.time()


Trial 20 Complete [00h 02m 40s]
val_loss: 0.08746278285980225

Best val_loss So Far: 0.08746278285980225
Total elapsed time: 00h 55m 47s


In [18]:
# Saving best HPs
best_hp = tuner.get_best_hyperparameters(1)[0]
hp_dict = {
    "patch_size":    best_hp.get("patch_size"),
    "hidden_dim":    best_hp.get("hidden_dim"),
    "dropout":       best_hp.get("dropout"),
    "learning_rate": best_hp.get("learning_rate"),
}
with open(HP_PATH, "w") as f:
    json.dump(hp_dict, f, indent=4)
print(f"[{WIN_KEY}] Best HPs saved → {HP_PATH}")

[48to12] Best HPs saved → /content/drive/MyDrive/EdgeMeter_AIv2/data/best_liteformer_hp_48to12.json


In [19]:
# Log
best_model = tuner.get_best_models(1)[0]
platform_info = platform.platform()
tune_log = {
    "window": WIN_KEY,
    "model": "LiteFormerV1",
    "task": "Smart Meter Energy Forecasting",
    "tuning_type": "BayesianOptimization",
    "subset_frac": subset_frac,
    "timesteps": int(timesteps),
    "n_features": int(n_features),
    "out_steps": int(out_steps),
    "tuning_time_minutes": round((end_time - start_time) / 60, 2),
    "best_hyperparameters": hp_dict,
    "total_params": int(best_model.count_params()),
    "input_shape": list(X_train.shape[1:]),
    "sequence_length": int(X_train.shape[1]),
    "gpu_used": gpu_name,
    "platform": platform_info,
    "log_type": "Tuning"
}
with open(LOG_PATH, "w") as f:
    json.dump(tune_log, f, indent=4)

print(f"[{WIN_KEY}] Tuning log → {LOG_PATH}")
print("LiteFormer v2 tuning (with v1 architecture) complete.")

[48to12] Tuning log → /content/drive/MyDrive/EdgeMeter_AIv2/data/liteformer_tuning_log_48to12.json
LiteFormer v2 tuning (with v1 architecture) complete.


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
